# Annualise Nino 3.4 data for mean and variance
Take monthly indices and generate annual averages

## Load data from `.csv`

In [1]:
import pandas as pd
import glob

In [2]:
dataname = 'nino34'

In [3]:
fname, = glob.glob(f"../data/{dataname}.csv")
df = pd.read_csv(fname)
df.head()

,Year,1,2,3,4,5,6,7,8,9,10,11,12
0,1950,-1.99,-1.69,-1.42,-1.54,-1.75,-1.27,-1.01,-0.97,-0.98,-1.03,-1.23,-1.31
1,1951,-1.30,-1.04,-0.38,-0.23,-0.01,0.00,0.30,0.17,0.51,0.49,0.55,0.31
2,1952,0.13,-0.01,-0.11,-0.02,-0.14,-0.54,-0.76,-0.56,-0.36,-0.46,-0.78,-0.39
3,1953,0.20,0.24,0.29,0.22,0.35,0.39,0.14,0.09,0.29,0.16,0.18,0.41
4,1954,0.43,0.28,-0.38,-1.17,-0.81,-0.93,-1.18,-1.43,-1.60,-1.48,-1.13,-1.33


In [4]:
df.tail()

,Year,1,2,3,4,5,6,7,8,9,10,11,12
67,2017,-0.43,-0.08,0.03,0.22,0.37,0.34,0.25,-0.16,-0.43,-0.56,-0.97,-0.98
68,2018,-0.98,-0.78,-0.80,-0.51,-0.20,0.04,0.12,0.09,0.47,0.90,0.90,0.89
69,2019,0.65,0.71,0.81,0.62,0.55,0.45,0.35,0.04,0.03,0.48,0.52,0.52
70,2020,0.60,0.37,0.48,0.36,-0.27,-0.34,-0.30,-0.59,-0.83,-1.26,-1.42,-1.15
71,2021,-1.00,-1.00,-0.80,-0.71,-0.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Longify

In [5]:
dfl = df.melt(id_vars=['Year'], value_vars=map(str, range(1, 13)), var_name='month', value_name=dataname)

## Remove null values

In [6]:
dfl.dropna(inplace=True)

In [7]:
dfl.tail()

,Year,month,nino34
858,2016,12,-0.50
859,2017,12,-0.98
860,2018,12,0.89
861,2019,12,0.52
862,2020,12,-1.15


## Timey Wimey stuff

In [8]:
dfl['year'] = dfl['Year']

In [9]:
dfl['month'] = pd.to_numeric(dfl['month'])

In [10]:
dfl['date'] = pd.to_datetime(dfl.year.astype(str)+dfl.month.astype(str), format='%Y%m')

In [11]:
dfl

,Year,month,nino34,year,date
0,1950,1,-1.99,1950,1950-01-01
1,1951,1,-1.30,1951,1951-01-01
2,1952,1,0.13,1952,1952-01-01
3,1953,1,0.20,1953,1953-01-01
4,1954,1,0.43,1954,1954-01-01
...,...,...,...,...,...
858,2016,12,-0.50,2016,2016-12-01
859,2017,12,-0.98,2017,2017-12-01
860,2018,12,0.89,2018,2018-12-01
861,2019,12,0.52,2019,2019-12-01


In [12]:
dfl['north'] = dfl['date'].dt.to_period('A-NOV')

In [13]:
dfl['south'] = dfl['date'].dt.to_period('A-MAY')

In [14]:
dfl['tropic'] = dfl['date'].dt.to_period('A-APR')

In [15]:
dfl

,Year,month,nino34,year,date,north,south,tropic
0,1950,1,-1.99,1950,1950-01-01,1950,1950,1950
1,1951,1,-1.30,1951,1951-01-01,1951,1951,1951
2,1952,1,0.13,1952,1952-01-01,1952,1952,1952
3,1953,1,0.20,1953,1953-01-01,1953,1953,1953
4,1954,1,0.43,1954,1954-01-01,1954,1954,1954
...,...,...,...,...,...,...,...,...
858,2016,12,-0.50,2016,2016-12-01,2017,2017,2017
859,2017,12,-0.98,2017,2017-12-01,2018,2018,2018
860,2018,12,0.89,2018,2018-12-01,2019,2019,2019
861,2019,12,0.52,2019,2019-12-01,2020,2020,2020


## Aggregate - Mean

In [16]:
def mean_to_csv(var, dataname=dataname):
    (dfl[[var, dataname]]
     .rename(columns={var: 'year', dataname: f"{dataname}_mean_{var}"})
     .groupby('year')
     .mean()
     .to_csv(f'../data/{dataname}_annual_mean_{var}.csv'))

Northern Hemisphere (Dec-Dec)

In [17]:
mean_to_csv('north')

Southern Hemisphere (Jun-Jun)

In [18]:
mean_to_csv('south')

Tropical year (May-May)

In [19]:
mean_to_csv('tropic')

Hsiang 2011 - "Civil conflicts are associated with the global climate" (Mar-Dec) <br>
https://www.nature.com/articles/nature10311

In [20]:
(dfl[dfl.month > 4]
 .groupby('year')
 .mean()[dataname]
 .to_csv(f'../data/{dataname}_annual_mean_hsiang2011.csv')
)

## Aggregate - Variance

In [21]:
def variance_to_csv(var, dataname=dataname):
    (dfl[[var, dataname]]
     .rename(columns={var: 'year', dataname: f"{dataname}_variance_{var}"})
     .groupby('year')
     .var()
     .to_csv(f'../data/{dataname}_annual_variance_{var}.csv'))

Northern Hemisphere (Dec-Dec)

In [22]:
variance_to_csv('north')

Southern Hemisphere (Jun-Jun)

In [23]:
variance_to_csv('south')

Tropical year (May-May)

In [24]:
variance_to_csv('tropic')

Hsiang 2011 - "Civil conflicts are associated with the global climate" (Mar-Dec) <br>
https://www.nature.com/articles/nature10311

In [25]:
(dfl[dfl.month > 4]
 .groupby('year')
 .var()[dataname]
 .to_csv(f'../data/{dataname}_annual_variance_hsiang2011.csv')
)